# Anna KaRNNa

In this notebook, I'll build a character-wise RNN trained on Anna Karenina, one of my all-time favorite books. It'll be able to generate new text based on the text from the book.

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Also, some information [here at r2rt](http://r2rt.com/recurrent-neural-networks-in-tensorflow-ii.html) and from [Sherjil Ozair](https://github.com/sherjilozair/char-rnn-tensorflow) on GitHub. Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

In [16]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

First we'll load the text file and convert it into integers for our network to use. Here I'm creating a couple dictionaries to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [17]:
with open('anna.txt', 'r') as f:
    text=f.read()
vocab = sorted(set(text))
print(vocab)
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

['\n', ' ', '!', '"', '$', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [18]:
vocab_to_int['h']

64

In [19]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

And we can see the characters encoded as integers.

Since the network is working with individual characters, it's similar to a classification problem in which we are trying to predict the next character from the previous text.  Here's how many 'classes' our network has to pick from.

In [20]:
len(vocab)

83

In [21]:
encoded[:10]

array([31, 64, 57, 72, 76, 61, 74,  1, 16,  0])

In [22]:
encoded.shape  # total characters in text

(1985223,)

## Making training mini-batches

Here is where we'll make our mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="assets/sequence_batching@1x.png" width=500px>


<br>
We have our text encoded as integers as one long array in `encoded`. Let's create a function that will give us an iterator for our batches. I like using [generator functions](https://jeffknupp.com/blog/2013/04/07/improve-your-python-yield-and-generators-explained/) to do this. Then we can pass `encoded` into this function and get our batch generator.

The first thing we need to do is discard some of the text so we only have completely full batches. Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences) and $M$ is the number of steps. Then, to get the number of batches we can make from some array `arr`, you divide the length of `arr` by the batch size. Once you know the number of batches and the batch size, you can get the total number of characters to keep.

After that, we need to split `arr` into $N$ sequences. You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences (`n_seqs` below), let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$ where $K$ is the number of batches.

Now that we have this array, we can iterate through it to get our batches. The idea is each batch is a $N \times M$ window on the array. For each subsequent batch, the window moves over by `n_steps`. We also want to create both the input and target arrays. Remember that the targets are the inputs shifted over one character. You'll usually see the first input character used as the last target character, so something like this:
```python
y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
```
where `x` is the input batch and `y` is the target batch.

The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of steps in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `n_steps` wide.

In [23]:
def get_batches(arr, n_seqs, n_steps):
    '''Create a generator that returns batches of size
       n_seqs x n_steps from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       n_seqs: Batch size, the number of sequences per batch
       n_steps: Number of sequence steps per batch
    '''
    # Get the number of characters per batch and number of batches we can make
    characters_per_batch = n_seqs * n_steps
    n_batches = len(arr)//characters_per_batch
    print('total number of batches :', n_batches)
    print('batch_size:', n_seqs)
    print('Number of sequence steps per batch:', n_steps)
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * characters_per_batch]
    
    # Reshape into n_seqs rows
    arr = arr.reshape((n_seqs, -1))
    
    for n in range(0, arr.shape[1], n_steps):
        # The features
        x = arr[:, n:n+n_steps]
        # The targets, shifted by one
        y = np.zeros_like(x)
        y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
        yield x, y

Now I'll make my data sets and we can check out what's going on here. Here I'm going to use a batch size of 10 and 50 sequence steps.

In [24]:
batches = get_batches(encoded, 10, 50)
x, y = next(batches)
x.shape

total number of batches : 3970
batch_size: 10
Number of sequence steps per batch: 50


(10, 50)

In [25]:
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[31 64 57 72 76 61 74  1 16  0]
 [ 1 57 69  1 70 71 76  1 63 71]
 [78 65 70 13  0  0  3 53 61 75]
 [70  1 60 77 74 65 70 63  1 64]
 [ 1 65 76  1 65 75 11  1 75 65]
 [ 1 37 76  1 79 57 75  0 71 70]
 [64 61 70  1 59 71 69 61  1 62]
 [26  1 58 77 76  1 70 71 79  1]
 [76  1 65 75 70  7 76 13  1 48]
 [ 1 75 57 65 60  1 76 71  1 64]]

y
 [[64 57 72 76 61 74  1 16  0  0]
 [57 69  1 70 71 76  1 63 71 65]
 [65 70 13  0  0  3 53 61 75 11]
 [ 1 60 77 74 65 70 63  1 64 65]
 [65 76  1 65 75 11  1 75 65 74]
 [37 76  1 79 57 75  0 71 70 68]
 [61 70  1 59 71 69 61  1 62 71]
 [ 1 58 77 76  1 70 71 79  1 75]
 [ 1 65 75 70  7 76 13  1 48 64]
 [75 57 65 60  1 76 71  1 64 61]]


If you implemented `get_batches` correctly, the above output should look something like 
```
x
 [[55 63 69 22  6 76 45  5 16 35]
 [ 5 69  1  5 12 52  6  5 56 52]
 [48 29 12 61 35 35  8 64 76 78]
 [12  5 24 39 45 29 12 56  5 63]
 [ 5 29  6  5 29 78 28  5 78 29]
 [ 5 13  6  5 36 69 78 35 52 12]
 [63 76 12  5 18 52  1 76  5 58]
 [34  5 73 39  6  5 12 52 36  5]
 [ 6  5 29 78 12 79  6 61  5 59]
 [ 5 78 69 29 24  5  6 52  5 63]]

y
 [[63 69 22  6 76 45  5 16 35 35]
 [69  1  5 12 52  6  5 56 52 29]
 [29 12 61 35 35  8 64 76 78 28]
 [ 5 24 39 45 29 12 56  5 63 29]
 [29  6  5 29 78 28  5 78 29 45]
 [13  6  5 36 69 78 35 52 12 43]
 [76 12  5 18 52  1 76  5 58 52]
 [ 5 73 39  6  5 12 52 36  5 78]
 [ 5 29 78 12 79  6 61  5 59 63]
 [78 69 29 24  5  6 52  5 63 76]]
 ```
 although the exact numbers will be different. Check to make sure the data is shifted over one step for `y`.

## Building the model

Below is where you'll build the network. We'll break it up into parts so it's easier to reason about each bit. Then we can connect them up into the whole network.

<img src="assets/charRNN.png" width=500px>


### Inputs

First off we'll create our input placeholders. As usual we need placeholders for the training data and the targets. We'll also create a placeholder for dropout layers called `keep_prob`.

In [26]:
def build_inputs(batch_size, num_steps):
    ''' Define placeholders for inputs, targets, and dropout 
    
        Arguments
        ---------
        batch_size: Batch size, number of sequences per batch
        num_steps: Number of sequence steps in a batch
        
    '''
    # Declare placeholders we'll feed into the graph
    inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name='inputs')
    targets = tf.placeholder(tf.int32, [batch_size, num_steps], name='targets')
    
    # Keep probability placeholder for drop out layers
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    return inputs, targets, keep_prob

### LSTM Cell

Here we will create the LSTM cell we'll use in the hidden layer. We'll use this cell as a building block for the RNN. So we aren't actually defining the RNN here, just the type of cell we'll use in the hidden layer.

We first create a basic LSTM cell with

```python
lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
```

where `num_units` is the number of units in the hidden layers in the cell. Then we can add dropout by wrapping it with 

```python
tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
```
You pass in a cell and it will automatically add dropout to the inputs or outputs. Finally, we can stack up the LSTM cells into layers with [`tf.contrib.rnn.MultiRNNCell`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/rnn/MultiRNNCell). With this, you pass in a list of cells and it will send the output of one cell into the next cell. Previously with TensorFlow 1.0, you could do this

```python
tf.contrib.rnn.MultiRNNCell([cell]*num_layers)
```

This might look a little weird if you know Python well because this will create a list of the same `cell` object. However, TensorFlow 1.0 will create different weight matrices for all `cell` objects. But, starting with TensorFlow 1.1 you actually need to create new cell objects in the list. To get it to work in TensorFlow 1.1, it should look like

```python
def build_cell(num_units, keep_prob):
    lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    return drop
    
tf.contrib.rnn.MultiRNNCell([build_cell(num_units, keep_prob) for _ in range(num_layers)])
```

Even though this is actually multiple LSTM cells stacked on each other, you can treat the multiple layers as one cell.

We also need to create an initial cell state of all zeros. This can be done like so

```python
initial_state = cell.zero_state(batch_size, tf.float32)
```

Below, we implement the `build_lstm` function to create these LSTM cells and the initial state.

In [27]:
def build_lstm(lstm_size, num_layers, batch_size, keep_prob):
    ''' Build LSTM cell.
    
        Arguments
        ---------
        keep_prob: Scalar tensor (tf.placeholder) for the dropout keep probability
        lstm_size: Size of the hidden layers in the LSTM cells
        num_layers: Number of LSTM layers
        batch_size: Batch size

    '''
    ### Build the LSTM Cell
    print('lstm_size:', lstm_size)
    def build_cell(lstm_size, keep_prob):
        # Use a basic LSTM cell
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        
        # Add dropout to the cell
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop
    
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([build_cell(lstm_size, keep_prob) for _ in range(num_layers)])
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    return cell, initial_state

### RNN Output

Here we'll create the output layer. We need to connect the output of the RNN cells to a full connected layer with a softmax output. The softmax output gives us a probability distribution we can use to predict the next character.

If our input has batch size $N$, number of steps $M$, and the hidden layer has $L$ hidden units, then the output is a 3D tensor with size $N \times M \times L$. The output of each LSTM cell has size $L$, we have $M$ of them, one for each sequence step, and we have $N$ sequences. So the total size is $N \times M \times L$.

We are using the same fully connected layer, the same weights, for each of the outputs. Then, to make things easier, we should reshape the outputs into a 2D tensor with shape $(M * N) \times L$. That is, one row for each sequence and step, where the values of each row are the output from the LSTM cells.

One we have the outputs reshaped, we can do the matrix multiplication with the weights. We need to wrap the weight and bias variables in a variable scope with `tf.variable_scope(scope_name)` because there are weights being created in the LSTM cells. TensorFlow will throw an error if the weights created here have the same names as the weights created in the LSTM cells, which they will be default. To avoid this, we wrap the variables in a variable scope so we can give them unique names.

In [28]:
def build_output(lstm_output, in_size, out_size):
    ''' Build a softmax layer, return the softmax output and logits.
    
        Arguments
        ---------
        
        x: Input tensor
        in_size: Size of the input tensor, for example, size of the LSTM cells
        out_size: Size of this softmax layer
    
    '''
    print('shape of lstm output:', lstm_output.get_shape())
    print('size of the LSTM cells:', in_size)
    print('size of softmax layer:', out_size)
    # Reshape output so it's a bunch of rows, one row for each step for each sequence.
    # That is, the shape should be batch_size*num_steps rows by lstm_size columns
    seq_output = tf.concat(lstm_output, axis=1)
    print('seq_output shape:', seq_output.get_shape())
    x = tf.reshape(seq_output, [-1, in_size])
    
    # Connect the RNN outputs to a softmax layer
    with tf.variable_scope('softmax'):
        softmax_w = tf.Variable(tf.truncated_normal((in_size, out_size), stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(out_size))
    #softmax_w = tf.Variable(tf.truncated_normal((in_size, out_size), stddev=0.1))
    #softmax_b = tf.Variable(tf.zeros(out_size))
        
    # Since output is a bunch of rows of RNN cell outputs, logits will be a bunch
    # of rows of logit outputs, one for each step and sequence
    logits = tf.matmul(x, softmax_w) + softmax_b
    
    # Use softmax to get the probabilities for predicted characters
    out = tf.nn.softmax(logits, name='predictions')
    
    return out, logits

### Training loss

Next up is the training loss. We get the logits and targets and calculate the softmax cross-entropy loss. First we need to one-hot encode the targets, we're getting them as encoded characters. Then, reshape the one-hot targets so it's a 2D tensor with size $(M*N) \times C$ where $C$ is the number of classes/characters we have. Remember that we reshaped the LSTM outputs and ran them through a fully connected layer with $C$ units. So our logits will also have size $(M*N) \times C$.

Then we run the logits and targets through `tf.nn.softmax_cross_entropy_with_logits` and find the mean to get the loss.

In [29]:
def build_loss(logits, targets, lstm_size, num_classes):
    ''' Calculate the loss from the logits and the targets.
    
        Arguments
        ---------
        logits: Logits from final fully connected layer
        targets: Targets for supervised learning
        lstm_size: Number of LSTM hidden units
        num_classes: Number of classes in targets
        
    '''
    print('logits shape:', logits.get_shape())
    # One-hot encode targets and reshape to match logits, one row per batch_size per step
    y_one_hot = tf.one_hot(targets, num_classes)
    y_reshaped = tf.reshape(y_one_hot, logits.get_shape())
    
    # Softmax cross entropy loss
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
    loss = tf.reduce_mean(loss)
    return loss

### Optimizer

Here we build the optimizer. Normal RNNs have have issues gradients exploding and disappearing. LSTMs fix the disappearance problem, but the gradients can still grow without bound. To fix this, we can clip the gradients above some threshold. That is, if a gradient is larger than that threshold, we set it to the threshold. This will ensure the gradients never grow overly large. Then we use an AdamOptimizer for the learning step.

In [30]:
def build_optimizer(loss, learning_rate, grad_clip):
    ''' Build optmizer for training, using gradient clipping.
    
        Arguments:
        loss: Network loss
        learning_rate: Learning rate for optimizer
    
    '''
    
    # Optimizer for training, using gradient clipping to control exploding gradients
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

### Build the network

Now we can put all the pieces together and build a class for the network. To actually run data through the LSTM cells, we will use [`tf.nn.dynamic_rnn`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/nn/dynamic_rnn). This function will pass the hidden and cell states across LSTM cells appropriately for us. It returns the outputs for each LSTM cell at each step for each sequence in the mini-batch. It also gives us the final LSTM state. We want to save this state as `final_state` so we can pass it to the first LSTM cell in the the next mini-batch run. For `tf.nn.dynamic_rnn`, we pass in the cell and initial state we get from `build_lstm`, as well as our input sequences. Also, we need to one-hot encode the inputs before going into the RNN. 

In [31]:
class CharRNN:
    
    def __init__(self, num_classes, batch_size=64, num_steps=50, 
                       lstm_size=128, num_layers=2, learning_rate=0.001, 
                       grad_clip=5, sampling=False):
        
        print('num of class:', num_classes)
        # When we're using this network for sampling later, we'll be passing in
        # one character at a time, so providing an option for that
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps

        tf.reset_default_graph()
        
        # Build the input placeholder tensors
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps)

        # Build the LSTM cell
        cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)

        ### Run the data through the RNN layers
        # First, one-hot encode the input tokens
        x_one_hot = tf.one_hot(self.inputs, num_classes)      
         # shape of self.inputs is: batch_size * num_steps, x_one_hot  is : batch_size * num_steps * num_classes after tf.one_hot #
            
        print('CharRNN input shape:', self.inputs.get_shape())
        print('CharRNN input one hot shape', x_one_hot.get_shape())
        # Run each sequence step through the RNN and collect the outputs
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=self.initial_state)
        #outputs size is: batch_size X num_steps X  lstm_size
        self.final_state = state
        
        # Get softmax predictions and logits
        self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)
        
        # Loss and optimizer (with gradient clipping)
        self.loss = build_loss(self.logits, self.targets, lstm_size, num_classes)
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)

>What I'm very confused is the batch shape(2D), but the above code cell
```python 
x_one_hot = tf.one_hot(self.inputs, num_classes) 
```
change 2D into 3D using one-hot enconder  (100,100) ----> (100,100, 83)

## Hyperparameters

Here I'm defining the hyperparameters for the network. 

* `batch_size` - Number of sequences running through the network in one pass.
* `num_steps` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lstm_size` - The number of units in the hidden layers.
* `num_layers` - Number of hidden LSTM layers to use
* `learning_rate` - Learning rate for training
* `keep_prob` - The dropout keep probability when training. If you're network is overfitting, try decreasing this.

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `lstm_size` and `num_layers`. I would advise that you always use `num_layers` of either 2/3. The `lstm_size` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `lstm_size` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.


In [32]:
batch_size = 100        # Sequences per batch
num_steps = 100         # Number of sequence steps per batch
lstm_size = 512         # Size of hidden layers in LSTMs
num_layers = 2          # Number of LSTM layers
learning_rate = 0.001   # Learning rate
keep_prob = 0.5         # Dropout keep probability

## Time for training

This is typical training code, passing inputs and targets into the network, then running the optimizer. Here we also get back the final LSTM state for the mini-batch. Then, we pass that state back into the network so the next batch can continue the state from the previous batch. And every so often (set by `save_every_n`) I save a checkpoint.

Here I'm saving checkpoints with the format

`i{iteration number}_l{# hidden layer units}.ckpt`

In [17]:
epochs = 20
# Save every N iterations
save_every_n = 200

model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/______.ckpt')
    counter = 0
    for e in range(epochs):
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(encoded, batch_size, num_steps):
            print('x batch in shape:', x.shape)
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            
            end = time.time()
            print('Epoch: {}/{}... '.format(e+1, epochs),
                  'Training Step: {}... '.format(counter),
                  'Training loss: {:.4f}... '.format(batch_loss),
                  '{:.4f} sec/batch'.format((end-start)))
        
            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
    
    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

num of class: 83
lstm_size: 512
CharRNN input shape: (100, 100)
CharRNN input one hot shape (100, 100, 83)
shape of lstm output: (100, 100, 512)
size of the LSTM cells: 512
size of softmax layer: 83
seq_output shape: (100, 100, 512)
logits shape: (10000, 83)
total number of batches : 198
batch_size: 100
Number of sequence steps per batch: 100
x batch in shape: (100, 100)
Epoch: 1/20...  Training Step: 1...  Training loss: 4.4209...  7.7064 sec/batch
x batch in shape: (100, 100)
Epoch: 1/20...  Training Step: 2...  Training loss: 4.3386...  7.8845 sec/batch
x batch in shape: (100, 100)
Epoch: 1/20...  Training Step: 3...  Training loss: 3.9185...  7.7440 sec/batch
x batch in shape: (100, 100)
Epoch: 1/20...  Training Step: 4...  Training loss: 5.2110...  7.0836 sec/batch
x batch in shape: (100, 100)
Epoch: 1/20...  Training Step: 5...  Training loss: 4.1636...  7.0769 sec/batch
x batch in shape: (100, 100)
Epoch: 1/20...  Training Step: 6...  Training loss: 3.7394...  7.3002 sec/batch
x

Epoch: 1/20...  Training Step: 73...  Training loss: 3.1239...  8.5436 sec/batch
x batch in shape: (100, 100)
Epoch: 1/20...  Training Step: 74...  Training loss: 3.1315...  8.7002 sec/batch
x batch in shape: (100, 100)
Epoch: 1/20...  Training Step: 75...  Training loss: 3.1332...  7.6770 sec/batch
x batch in shape: (100, 100)
Epoch: 1/20...  Training Step: 76...  Training loss: 3.1363...  7.0325 sec/batch
x batch in shape: (100, 100)
Epoch: 1/20...  Training Step: 77...  Training loss: 3.1340...  7.3503 sec/batch
x batch in shape: (100, 100)
Epoch: 1/20...  Training Step: 78...  Training loss: 3.1227...  7.4508 sec/batch
x batch in shape: (100, 100)
Epoch: 1/20...  Training Step: 79...  Training loss: 3.1183...  7.1596 sec/batch
x batch in shape: (100, 100)
Epoch: 1/20...  Training Step: 80...  Training loss: 3.1037...  7.3432 sec/batch
x batch in shape: (100, 100)
Epoch: 1/20...  Training Step: 81...  Training loss: 3.1074...  6.9685 sec/batch
x batch in shape: (100, 100)
Epoch: 1/2

Epoch: 1/20...  Training Step: 148...  Training loss: 3.0309...  7.7575 sec/batch
x batch in shape: (100, 100)
Epoch: 1/20...  Training Step: 149...  Training loss: 2.9918...  7.7851 sec/batch
x batch in shape: (100, 100)
Epoch: 1/20...  Training Step: 150...  Training loss: 3.0025...  7.1736 sec/batch
x batch in shape: (100, 100)
Epoch: 1/20...  Training Step: 151...  Training loss: 3.0300...  7.1213 sec/batch
x batch in shape: (100, 100)
Epoch: 1/20...  Training Step: 152...  Training loss: 3.0231...  7.3435 sec/batch
x batch in shape: (100, 100)
Epoch: 1/20...  Training Step: 153...  Training loss: 2.9836...  8.4055 sec/batch
x batch in shape: (100, 100)
Epoch: 1/20...  Training Step: 154...  Training loss: 2.9855...  8.6973 sec/batch
x batch in shape: (100, 100)
Epoch: 1/20...  Training Step: 155...  Training loss: 2.9624...  7.9336 sec/batch
x batch in shape: (100, 100)
Epoch: 1/20...  Training Step: 156...  Training loss: 2.9661...  7.7185 sec/batch
x batch in shape: (100, 100)
E

Epoch: 2/20...  Training Step: 222...  Training loss: 2.5398...  9.4212 sec/batch
x batch in shape: (100, 100)
Epoch: 2/20...  Training Step: 223...  Training loss: 2.5274...  9.1410 sec/batch
x batch in shape: (100, 100)
Epoch: 2/20...  Training Step: 224...  Training loss: 2.5277...  8.7147 sec/batch
x batch in shape: (100, 100)
Epoch: 2/20...  Training Step: 225...  Training loss: 2.5246...  8.6817 sec/batch
x batch in shape: (100, 100)
Epoch: 2/20...  Training Step: 226...  Training loss: 2.5230...  8.5216 sec/batch
x batch in shape: (100, 100)
Epoch: 2/20...  Training Step: 227...  Training loss: 2.5395...  8.0278 sec/batch
x batch in shape: (100, 100)
Epoch: 2/20...  Training Step: 228...  Training loss: 2.5226...  8.0502 sec/batch
x batch in shape: (100, 100)
Epoch: 2/20...  Training Step: 229...  Training loss: 2.5353...  8.1763 sec/batch
x batch in shape: (100, 100)
Epoch: 2/20...  Training Step: 230...  Training loss: 2.5003...  7.9712 sec/batch
x batch in shape: (100, 100)
E

Epoch: 2/20...  Training Step: 296...  Training loss: 2.3301...  6.8298 sec/batch
x batch in shape: (100, 100)
Epoch: 2/20...  Training Step: 297...  Training loss: 2.3242...  6.8655 sec/batch
x batch in shape: (100, 100)
Epoch: 2/20...  Training Step: 298...  Training loss: 2.3155...  6.9213 sec/batch
x batch in shape: (100, 100)
Epoch: 2/20...  Training Step: 299...  Training loss: 2.3454...  7.0834 sec/batch
x batch in shape: (100, 100)
Epoch: 2/20...  Training Step: 300...  Training loss: 2.3322...  7.1137 sec/batch
x batch in shape: (100, 100)
Epoch: 2/20...  Training Step: 301...  Training loss: 2.3139...  7.0466 sec/batch
x batch in shape: (100, 100)
Epoch: 2/20...  Training Step: 302...  Training loss: 2.3093...  6.9594 sec/batch
x batch in shape: (100, 100)
Epoch: 2/20...  Training Step: 303...  Training loss: 2.3160...  6.9628 sec/batch
x batch in shape: (100, 100)
Epoch: 2/20...  Training Step: 304...  Training loss: 2.3306...  6.8160 sec/batch
x batch in shape: (100, 100)
E

...

Epoch: 19/20...  Training Step: 3660...  Training loss: 1.2449...  6.9965 sec/batch
x batch in shape: (100, 100)
Epoch: 19/20...  Training Step: 3661...  Training loss: 1.2410...  7.1177 sec/batch
x batch in shape: (100, 100)
Epoch: 19/20...  Training Step: 3662...  Training loss: 1.2080...  7.1462 sec/batch
x batch in shape: (100, 100)
Epoch: 19/20...  Training Step: 3663...  Training loss: 1.2332...  7.0727 sec/batch
x batch in shape: (100, 100)
Epoch: 19/20...  Training Step: 3664...  Training loss: 1.2257...  7.1327 sec/batch
x batch in shape: (100, 100)
Epoch: 19/20...  Training Step: 3665...  Training loss: 1.2477...  7.1858 sec/batch
x batch in shape: (100, 100)
Epoch: 19/20...  Training Step: 3666...  Training loss: 1.2278...  7.0907 sec/batch
x batch in shape: (100, 100)
Epoch: 19/20...  Training Step: 3667...  Training loss: 1.2460...  6.9528 sec/batch
x batch in shape: (100, 100)
Epoch: 19/20...  Training Step: 3668...  Training loss: 1.2335...  7.0546 sec/batch
x batch in s

Epoch: 19/20...  Training Step: 3733...  Training loss: 1.2554...  7.2103 sec/batch
x batch in shape: (100, 100)
Epoch: 19/20...  Training Step: 3734...  Training loss: 1.2423...  7.2637 sec/batch
x batch in shape: (100, 100)
Epoch: 19/20...  Training Step: 3735...  Training loss: 1.2250...  7.4212 sec/batch
x batch in shape: (100, 100)
Epoch: 19/20...  Training Step: 3736...  Training loss: 1.2386...  7.2708 sec/batch
x batch in shape: (100, 100)
Epoch: 19/20...  Training Step: 3737...  Training loss: 1.2674...  7.0091 sec/batch
x batch in shape: (100, 100)
Epoch: 19/20...  Training Step: 3738...  Training loss: 1.2364...  7.0380 sec/batch
x batch in shape: (100, 100)
Epoch: 19/20...  Training Step: 3739...  Training loss: 1.2419...  7.0020 sec/batch
x batch in shape: (100, 100)
Epoch: 19/20...  Training Step: 3740...  Training loss: 1.2122...  7.2156 sec/batch
x batch in shape: (100, 100)
Epoch: 19/20...  Training Step: 3741...  Training loss: 1.2406...  8.0903 sec/batch
x batch in s

Epoch: 20/20...  Training Step: 3805...  Training loss: 1.2274...  6.8421 sec/batch
x batch in shape: (100, 100)
Epoch: 20/20...  Training Step: 3806...  Training loss: 1.2060...  6.7715 sec/batch
x batch in shape: (100, 100)
Epoch: 20/20...  Training Step: 3807...  Training loss: 1.2426...  6.8169 sec/batch
x batch in shape: (100, 100)
Epoch: 20/20...  Training Step: 3808...  Training loss: 1.2219...  6.8330 sec/batch
x batch in shape: (100, 100)
Epoch: 20/20...  Training Step: 3809...  Training loss: 1.2207...  7.8556 sec/batch
x batch in shape: (100, 100)
Epoch: 20/20...  Training Step: 3810...  Training loss: 1.2308...  7.9905 sec/batch
x batch in shape: (100, 100)
Epoch: 20/20...  Training Step: 3811...  Training loss: 1.2229...  7.0106 sec/batch
x batch in shape: (100, 100)
Epoch: 20/20...  Training Step: 3812...  Training loss: 1.2507...  7.3107 sec/batch
x batch in shape: (100, 100)
Epoch: 20/20...  Training Step: 3813...  Training loss: 1.2021...  7.3673 sec/batch
x batch in s

Epoch: 20/20...  Training Step: 3878...  Training loss: 1.1953...  6.8489 sec/batch
x batch in shape: (100, 100)
Epoch: 20/20...  Training Step: 3879...  Training loss: 1.2346...  6.8169 sec/batch
x batch in shape: (100, 100)
Epoch: 20/20...  Training Step: 3880...  Training loss: 1.2394...  7.2725 sec/batch
x batch in shape: (100, 100)
Epoch: 20/20...  Training Step: 3881...  Training loss: 1.2254...  6.9986 sec/batch
x batch in shape: (100, 100)
Epoch: 20/20...  Training Step: 3882...  Training loss: 1.2261...  6.9078 sec/batch
x batch in shape: (100, 100)
Epoch: 20/20...  Training Step: 3883...  Training loss: 1.2271...  6.9280 sec/batch
x batch in shape: (100, 100)
Epoch: 20/20...  Training Step: 3884...  Training loss: 1.1969...  7.1439 sec/batch
x batch in shape: (100, 100)
Epoch: 20/20...  Training Step: 3885...  Training loss: 1.1913...  7.0366 sec/batch
x batch in shape: (100, 100)
Epoch: 20/20...  Training Step: 3886...  Training loss: 1.2251...  6.9610 sec/batch
x batch in s

Epoch: 20/20...  Training Step: 3951...  Training loss: 1.2167...  7.7782 sec/batch
x batch in shape: (100, 100)
Epoch: 20/20...  Training Step: 3952...  Training loss: 1.2153...  7.2528 sec/batch
x batch in shape: (100, 100)
Epoch: 20/20...  Training Step: 3953...  Training loss: 1.2068...  7.2382 sec/batch
x batch in shape: (100, 100)
Epoch: 20/20...  Training Step: 3954...  Training loss: 1.2155...  7.2103 sec/batch
x batch in shape: (100, 100)
Epoch: 20/20...  Training Step: 3955...  Training loss: 1.2281...  7.1167 sec/batch
x batch in shape: (100, 100)
Epoch: 20/20...  Training Step: 3956...  Training loss: 1.2141...  7.0022 sec/batch
x batch in shape: (100, 100)
Epoch: 20/20...  Training Step: 3957...  Training loss: 1.1911...  7.0856 sec/batch
x batch in shape: (100, 100)
Epoch: 20/20...  Training Step: 3958...  Training loss: 1.2321...  6.9980 sec/batch
x batch in shape: (100, 100)
Epoch: 20/20...  Training Step: 3959...  Training loss: 1.2248...  7.1326 sec/batch
x batch in s

#### Saved checkpoints

Read up on saving and loading checkpoints here: https://www.tensorflow.org/programmers_guide/variables

In [18]:
tf.train.get_checkpoint_state('checkpoints')

model_checkpoint_path: "checkpoints\\i3960_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i1000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i1200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i1400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i1600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i1800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i2000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i2200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i2400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i2600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i2800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i3000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i3200_l512.ckpt"
all_mo

## Sampling

Now that the network is trained, we'll can use it to generate new text. The idea is that we pass in a character, then the network will predict the next character. We can use the new one, to predict the next one. And we keep doing this to generate all new text. I also included some functionality to prime the network with some text by passing in a string and building up a state from that.

The network gives us predictions for each character. To reduce noise and make things a little less random, I'm going to only choose a new character from the top N most likely characters.



In [53]:
def pick_top_n(preds, vocab_size, top_n=5):
    print(preds.shape)          # (1, 83)
    print(vocab_size)
    p = np.squeeze(preds)   # (83, ) Remove single-dimensional entries from the shape
    #print(p)
    print(p.shape)
    p[np.argsort(p)[:-top_n]] = 0  # don't change the order in p, but assign 0 to element except of  top_n
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [50]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    samples = [c for c in prime]
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

Here, pass in the path to a checkpoint and sample from the network.

In [51]:
tf.train.latest_checkpoint('checkpoints')

'checkpoints\\i3960_l512.ckpt'

In [52]:
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="Far")
print(samp)

num of class: 83
lstm_size: 512
CharRNN input shape: (1, 1)
CharRNN input one hot shape (1, 1, 83)
shape of lstm output: (1, 1, 512)
size of the LSTM cells: 512
size of softmax layer: 83
seq_output shape: (1, 1, 512)
logits shape: (1, 83)
INFO:tensorflow:Restoring parameters from checkpoints\i3960_l512.ckpt
(1, 83)
83
[  1.79670251e-03   8.48622341e-03   1.35987252e-03   2.20882706e-04
   4.13246271e-05   3.67764151e-05   6.17338010e-05   1.96004738e-04
   3.06342699e-05   1.35946219e-04   5.39845169e-05   3.62909795e-03
   6.72275375e-04   1.73283659e-03   2.71370700e-05   7.98285400e-05
   1.34425572e-04   1.19585042e-04   8.53222373e-05   6.26871915e-05
   1.02372680e-04   5.93961668e-05   1.08500048e-04   7.99504487e-05
   8.37917396e-05   2.79472326e-04   8.61068605e-04   1.04662147e-03
   2.90205189e-05   2.92611745e-04   2.16551591e-04   1.45310332e-04
   1.37882831e-04   1.47966333e-04   2.23203766e-04   3.83134990e-04
   1.59632211e-04   2.72602920e-04   5.07905243e-05   1.499

(1, 83)
83
[  9.78669734e-04   2.70994633e-01   1.22248172e-03   3.58807913e-04
   9.50229122e-08   3.47803876e-07   1.76708468e-07   4.02063364e-04
   9.80787718e-07   4.52967288e-05   2.87713533e-07   4.09918800e-02
   5.57222543e-03   2.36100275e-02   1.95992627e-07   6.70488078e-07
   6.98849590e-07   2.82256821e-07   6.55270526e-07   3.56164207e-07
   1.06345027e-07   1.33007887e-07   1.94470914e-07   2.44622839e-07
   4.23320643e-07   6.37567195e-04   2.72443029e-03   2.69079668e-04
   7.43211999e-08   6.46818899e-06   6.43890189e-07   6.18363345e-07
   1.46951163e-06   4.29932840e-07   4.18075331e-07   1.24238667e-07
   1.46786726e-06   2.17745801e-05   9.49403045e-08   1.74485865e-06
   3.20890376e-06   1.52932887e-06   5.90709703e-07   4.52277050e-07
   5.73468242e-07   1.06767871e-07   4.93828765e-08   1.44025068e-06
   1.46957052e-06   9.40508471e-08   2.31367926e-06   1.21217897e-06
   3.39628116e-07   1.81965788e-06   1.12094696e-07   6.25058146e-06
   1.41602499e-07   7.4

(1, 83)
83
[  4.22677294e-05   8.79110652e-04   9.79177184e-06   7.18322553e-05
   5.14775067e-08   2.93363854e-07   1.80179924e-08   7.38081726e-05
   1.89450404e-07   4.81125880e-06   4.76810662e-08   2.50005774e-04
   6.42326340e-05   1.93511267e-04   2.26698134e-07   2.28708529e-07
   1.95940501e-07   2.14932328e-07   1.18952784e-07   5.96566139e-08
   2.71518470e-08   7.89504355e-08   1.43952363e-07   7.65612427e-08
   3.94098976e-08   2.08744837e-06   4.11277506e-05   1.12846401e-05
   1.87831919e-08   1.28403062e-05   2.19392018e-06   4.20687485e-07
   7.73188049e-07   5.30121667e-07   6.88463331e-07   5.32334343e-06
   1.37075619e-06   2.26512075e-05   3.75285367e-07   1.22066263e-06
   1.78857738e-06   6.59388934e-06   3.38627342e-06   1.09007249e-06
   2.81983921e-06   1.13356364e-07   7.14766747e-06   6.83663893e-06
   4.27204577e-06   2.55975863e-07   3.20876961e-06   2.01561352e-06
   1.10390104e-07   7.89815033e-07   8.04989142e-08   4.83321355e-06
   5.71721692e-08   9.2

(1, 83)
83
[  2.57470833e-06   1.86716003e-04   3.45256467e-07   1.54020802e-06
   3.68860886e-10   1.35517930e-09   1.08504650e-09   2.03731062e-07
   5.55885615e-10   5.58999282e-08   2.06286943e-09   8.28062639e-06
   1.44375713e-06   1.35060759e-06   4.31539915e-10   1.23771393e-09
   2.37260966e-09   1.82832025e-08   8.52790838e-10   4.03015066e-10
   1.22310517e-09   1.79258375e-09   1.63632730e-09   8.17017065e-10
   1.33629074e-09   9.35969098e-08   6.20305286e-07   8.23932169e-07
   3.02379649e-10   2.44891936e-07   7.73569440e-08   1.21509586e-07
   1.38132691e-07   1.27733273e-08   4.11387724e-08   2.83787838e-08
   5.13216136e-08   1.10242217e-06   2.42206011e-09   2.29925703e-07
   7.27805372e-08   6.13637155e-07   2.19237357e-08   5.67137661e-08
   1.24561112e-07   1.26558486e-09   9.93811078e-09   5.97130281e-07
   2.17246082e-07   1.06407452e-08   6.35537276e-08   3.27493240e-07
   2.85375751e-10   1.62545092e-07   1.02600106e-09   5.01984836e-08
   3.21427468e-10   1.7

(1, 83)
83
[  4.53956338e-04   1.76890765e-03   1.49360148e-05   5.61663292e-05
   1.37278903e-07   2.43657183e-07   2.11732612e-07   1.04039696e-04
   8.47920001e-06   3.32285163e-05   1.01498422e-07   2.13967622e-04
   1.14958995e-04   1.43243262e-04   2.93125140e-07   2.29184502e-06
   6.04559716e-07   7.21095034e-07   7.94461243e-07   3.45080934e-07
   3.15980486e-07   2.81654366e-07   4.21038010e-07   5.93097184e-07
   5.05244657e-07   4.10215080e-06   2.40612590e-05   8.53855090e-06
   1.22771397e-07   6.76303898e-06   7.96790300e-06   7.53298480e-07
   4.39419227e-06   1.16820229e-06   6.99351915e-07   3.45660169e-06
   8.65113088e-06   2.06374079e-05   1.79289873e-06   6.08583287e-06
   5.06312790e-06   4.01048874e-06   2.57051943e-06   1.05718004e-06
   3.80597680e-06   1.71132370e-07   2.18922264e-06   6.82810514e-06
   8.97859445e-06   2.11842598e-06   1.03857037e-05   1.01472006e-05
   7.71358089e-07   1.54893371e-06   4.95540860e-07   1.43074985e-05
   1.87255097e-07   2.4

...

(1, 83)
83
[  1.33585499e-03   4.80947971e-01   4.02195910e-05   1.99861330e-04
   9.43959044e-09   1.05200407e-07   7.44173576e-08   2.92243878e-03
   3.15879902e-07   1.59067549e-05   5.28280601e-08   3.38311703e-03
   3.11813463e-04   2.34718318e-03   8.67251018e-08   3.43387370e-07
   3.60934393e-07   1.03356591e-07   5.15841272e-08   5.66513236e-08
   7.10865677e-08   8.34909670e-08   8.88339784e-08   5.01118045e-08
   6.92136055e-08   1.37460502e-05   1.18919561e-04   4.25070612e-05
   1.28214843e-08   2.83106351e-06   8.32376372e-07   2.03874677e-07
   1.90491335e-07   2.18853074e-07   3.11196573e-07   1.72040416e-07
   5.26917347e-07   1.73375952e-06   9.31635640e-08   5.73635987e-07
   5.69323390e-07   4.41634768e-07   3.98546433e-07   3.99276473e-07
   4.82096368e-07   3.32651560e-08   1.47405956e-07   2.38279881e-06
   1.09820257e-06   2.00335052e-07   6.72936267e-07   1.48895754e-06
   5.13852747e-08   7.05073091e-07   3.91819519e-08   4.03788545e-05
   2.74873546e-08   2.4

(1, 83)
83
[  1.72954879e-03   8.46944213e-01   1.29049613e-06   1.56416754e-05
   3.35008465e-09   6.57849526e-08   4.15038004e-09   3.92215552e-05
   8.92441232e-09   1.24369008e-05   6.04411667e-08   7.96712760e-04
   3.06876827e-05   8.16786705e-05   1.66956340e-07   8.85814302e-08
   3.98810371e-08   1.90021971e-08   2.10057713e-08   1.98807335e-08
   1.22385435e-08   2.58263047e-08   2.54794053e-08   7.44738760e-09
   1.00394244e-08   9.78499202e-07   1.73949520e-05   2.24247651e-06
   3.93652222e-09   3.90253535e-06   5.99387135e-07   6.80010857e-08
   6.31443413e-08   8.00207047e-08   3.87323276e-08   2.51073715e-07
   2.79835064e-07   6.74307614e-07   1.97063166e-08   7.40994665e-07
   9.38530036e-07   7.13442603e-07   6.93262507e-07   4.28659135e-08
   3.04599638e-07   2.45087151e-09   1.95036179e-07   1.24180508e-06
   7.16619581e-07   4.72119019e-08   4.50644563e-07   1.74227193e-06
   6.91237112e-09   3.37165233e-07   6.60912836e-09   4.78290031e-06
   6.26128926e-09   2.8

(1, 83)
83
[  5.41644264e-03   9.06333804e-01   4.30216423e-05   4.53776374e-05
   1.87115390e-08   7.76388731e-08   2.64415068e-08   5.28967394e-05
   2.44130945e-07   1.28433703e-05   1.25563503e-07   5.94915031e-03
   6.20410021e-04   1.57600862e-03   6.40552784e-08   3.34089293e-07
   4.87530905e-08   1.35544781e-07   5.61175675e-08   3.75766582e-08
   6.20295850e-08   4.11122016e-08   5.29457154e-08   2.98445464e-08
   3.33515793e-08   4.28947678e-05   1.68373910e-04   5.50502955e-05
   9.00557495e-09   8.00657006e-07   3.77162650e-07   5.90967701e-08
   3.48964626e-08   2.05894167e-07   1.69928740e-07   1.30172253e-07
   1.62359882e-07   1.92544644e-06   7.61098562e-09   3.52800640e-07
   8.89256739e-08   3.85829651e-07   1.01367228e-07   1.46894678e-07
   3.76046216e-08   9.24075483e-09   1.15179102e-08   8.10322092e-07
   6.68139421e-07   9.49007770e-08   1.13829685e-07   1.70180192e-06
   1.51051065e-08   4.06176781e-07   1.85638331e-08   1.10566452e-06
   1.02206016e-07   1.0

(1, 83)
83
[  3.61936941e-06   4.54616995e-04   8.69258929e-06   1.79663621e-05
   4.53663738e-08   9.45439709e-08   5.02113977e-08   9.45743705e-06
   1.99338302e-07   1.81100961e-06   6.18289704e-08   9.13184485e-05
   1.35840373e-05   7.35802096e-05   3.21969367e-07   1.37176556e-07
   9.12689444e-08   8.17922015e-08   5.64694815e-08   6.94261999e-08
   2.19320544e-08   5.94959531e-08   3.95272366e-08   2.64619704e-08
   1.21912805e-07   2.99050384e-06   1.32035084e-05   9.75144303e-06
   8.21126420e-08   2.29182979e-06   1.01849389e-06   6.16831386e-08
   2.00003242e-07   2.18017504e-07   8.67150831e-08   1.75875499e-07
   5.16312014e-07   1.02953982e-06   1.68925041e-07   5.08303458e-07
   7.25344307e-07   5.69538372e-07   3.29216675e-07   4.89337197e-07
   6.76449361e-07   1.59168341e-08   6.11057487e-07   1.66572215e-06
   3.61694163e-07   2.97198905e-08   1.90436637e-07   1.34981531e-06
   4.41123902e-08   1.42568365e-06   7.82381875e-08   1.49693335e-06
   8.80233770e-08   4.4

KeyboardInterrupt: 

In [23]:
checkpoint = 'checkpoints/i200_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

num of class: 83
lstm_size: 512
CharRNN input shape: (1, 1)
CharRNN input one hot shape (1, 1, 83)
shape of lstm output: (1, 1, 512)
size of the LSTM cells: 512
size of softmax layer: 83
seq_output shape: (1, 1, 512)
logits shape: (1, 83)
INFO:tensorflow:Restoring parameters from checkpoints/i200_l512.ckpt
Fard

hins is hons, asd wo to ton has sonti than tas oe ton hh sintes are atet on ortens oto th ar on an in hin tins tase so int te teesen shis in tat in ing the th he hin hh sos har arthe the ant or ort ha tate sens oned or ane thin th asit ha andes, hi shis as on it he te sorsand ote tarisiti the serins ito and thiss ilind he tot ha tot he ward hins
isiton, tar oo ilig toe wh th sered ton tore
atot it oter ane an thit an thed, she teed at he sontas the hed she ton ho ared to he sos tot hor
ere shis sas ton tite seritit he was insit ine, wor the hin sont aritot hot wor oe tet as orot ar arisend an io hh is heress te orit hin ores he this altet an te hir orense ontan ha hir ond ans i

In [24]:
checkpoint = 'checkpoints/i600_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

num of class: 83
lstm_size: 512
CharRNN input shape: (1, 1)
CharRNN input one hot shape (1, 1, 83)
shape of lstm output: (1, 1, 512)
size of the LSTM cells: 512
size of softmax layer: 83
seq_output shape: (1, 1, 512)
logits shape: (1, 83)
INFO:tensorflow:Restoring parameters from checkpoints/i600_l512.ckpt
Farred," she shing had seand that she wald him allowe a merte and she sime to the comports, he
sanded and so her atay to be miment.

Shating the houg one with there her some and ate he sompess wish the sald of heare, and as to her that had ho wall ate as to the sarmed.

And the comen and so were, shead,
and a didention, who her stepe ald the wat san at the caring the chate seading time time it, at that
her had a ding tha didnent over the somper and the congout at their ande wite the parlioned the mention with the mane to tere her and when whe that he had had seed
the her she dad her soon," sould, hus some oun," and
wand the chorsedsed him southirg his and her and was not to hars oun 

In [25]:
checkpoint = 'checkpoints/i1200_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

num of class: 83
lstm_size: 512
CharRNN input shape: (1, 1)
CharRNN input one hot shape (1, 1, 83)
shape of lstm output: (1, 1, 512)
size of the LSTM cells: 512
size of softmax layer: 83
seq_output shape: (1, 1, 512)
logits shape: (1, 83)
INFO:tensorflow:Restoring parameters from checkpoints/i1200_l512.ckpt
Farlen's.

"What yin, I may, when they was not in it's a carriagion."

Stepan Arkadyevitch had be a sair and host of her asting," was the
plancess of the compreary of the same on the
complace. The post of his profections of the metter with the some was
thracking the prase, and the condictions, and
they wishand a see he was took as her and serviture of his fasher, with a pliget of his head, and that he was suchely and was seeming
too and the first what therse was still been to the stand of the past that
stepp to the world took a sheak word over him as her to seemen, and had been so a consentaor over the whice and terring
to the sense of stopily the surch of imeness.

"Wher a dear all